***'Syntax natürlicher Sprachen',*** **Wintersemester 2021/22**

--- 
# Klausur: Syntax natürlicher Sprachen *(15.02.2022)*

- **Bearbeitungszeitraum: 10:15-11:45**
- **Abgabe in Moodle bis 12:00**


## Übersicht

- [Hinweise zur Bearbeitung](#Hinweise-zur-Bearbeitung)
- [Laden von Paketen](#Laden-von-Paketen)


1. [Ambiguität in Konstituentengrammatik](#1.-Ambiguität-in-Konstituentengrammatik)
2. [Ambiguität in Dependenzgrammatik](#2.-Ambiguität-in-Dependenzgrammatik)
3. [Konstituenten- und Adjunkttests](#3.-Konstituenten--und-Adjunkttests)
4. [CFG Analysen](#4.-CFG-Analysen)
5. [Dependenzanalyse](#5.-Dependenzanalyse)
6. [FCFG-Erweiterung: Kasus, Agreement, Subkategorisierung](#6.-FCFG-Erweiterung:-Kasus,-Agreement,-Subkategorisierung)
7. [Slash-Kategorien, Inversion, Wortstellung](#7.-Slash-Kategorien,-Inversion,-Wortstellung)
8. [Syntaxregeln komplexer Sätze](#8.-Syntaxregeln-komplexer-Sätze)
9. [Parsing-Algorithmen und Rekursionstypen](#9.-Parsing-Algorithmen-und-Rekursionstypen)
10. [Unifikationsparsing](#10.-Unifikationsparsing-und-getypte-Merkmalstrukturen)
11. [Statistisches Parsing](#11.-Statistisches-Parsing)
12. [Datengestützte Syntaxanalyse](#12.-Datengestützte-Syntaxanalyse)
13. [Chunk-Analysen](#13.-Chunk-Analysen)



---

## Hinweise zur Bearbeitung

### Falls noch nicht geschehen, benennen Sie bitte zunächst die Datei der Klausur-Angabe nach folgendem Schema um: 

 #### `Nachname_Vorname_Matrikelnummer.ipynb`
 

### Geben Sie nun in folgender Codezelle die letzte Ziffer Ihrer Matrikelnummer ein (`last_number`) und führen Sie die Codezelle aus, um Ihre Gruppennummer zu berechnen:

In [ ]:
last_number = 

group_number = int((((last_number*3)%10)/3.5))+1
print("Ihre Gruppennummer: "+str(group_number))

---
## *WICHTIG: Bearbeiten Sie in den Aufgaben, in denen explizit darauf hingewiesen wird, nur die Aufgabenstellung, die Ihnen gemäß Ihrer Gruppennummer zugeordnet wird!*


--- 

## Hinweise zum Ausfüllen der Codezellen

### Wenn gegeben, führen Sie zunächst die mit `#RUN` markierten Codezellen zu Beginn einer Aufgabe aus (dies ist für eine erfolgreiche Bearbeitung der Aufgabe notwendig).

### Verändern Sie nur die `#TO_DO`-Codezellen (nur gemäß der Angabe in der jeweiligen Aufgabe)!

### Führen Sie `#TO_DO`-Codezellen nach Bearbeitung aus, um das Output ihrer Lösung zu generieren (dieses muss als Teil Ihrer Lösung mit abgespeichert werden); bei aufeinander aufbauenden Aufgaben (`a)`, `b)`usw.) ist zudem  notwendig, dass Sie Ihre Lösung aus der vorangehenden Teilaufgabe ausführen, damit diese in der folgenden zur Verfügung steht.

### Angegebene Inhalte (Grammatikregeln usw.) dürfen nicht auskommentiert oder gelöscht werden, außer dies wird explizit anders erwähnt!


### WICHTIG: Setzen Sie den Status des Notebooks ggf. auf `Trusted`, damit alle angegebenen Outputs korrekt angezeigt werden.

---

## Hinweis zur Bewertung

### *Jede korrekt gelöste Teilaufgabe (`#TO_DO`-Codezelle) wird mit 2 Punkten bewertet.*

---

[zurück zur Übersicht](#Übersicht)


---

## Laden von Paketen

### Führen Sie zu Beginn folgende Codezelle aus.

### Das erfolgreiche Ausführen dieser Codezelle ist Voraussetzung für die Bearbeitung der folgenden Aufgaben.  

In [1]:
#RUN (Führen Sie diese Code-Zelle aus:)
import nltk
from nltk.tree import Tree
from nltk import FeatStruct
import itertools


from spacy import displacy

def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split()
        line_list.pop(0)
        line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))

    return "\n".join([i for i in sent_list[0:]])



from nltk import DependencyGraph
from itertools import chain

def _tree_labeled(self, i):
        node = self.get_by_address(i)
        word = node["word"]
        rel = node["rel"]        
        deps = sorted(chain.from_iterable(node["deps"].values()))

        if deps:
            return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])
        else:
            return word+'('+rel+')'
        
def tree_labeled(self):
        node = self.root

        word = node["word"]
        rel = node["rel"]
        deps = sorted(chain.from_iterable(node["deps"].values()))
        return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled



def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())

    deps = [x for x in deps if x]

    ex = []
    word_list = []
    arc_list = []

    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1
        head = int(dep[2])
        label = dep[3]
        if head>line:
            start = index
            end = head-1
            direction = "left"
        else:
            start = head-1
            end = index  
            direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})

    ex.append({
        "words": word_list,
        "arcs": arc_list
    })    

    return ex



from nltk.featstruct import Feature, UnificationFailure, FeatStructReader
import itertools
from collections import defaultdict


def check_sanity_constraints(th):
    for type1, type2 in itertools.product(th, th):
        if type1 in th[type2] and type2 in th[type1]:
            if type1 != type2:
                raise ValueError(
                    "The type hierarchy is not antisymmetric! " +
                    "{} subsumes {} and vice versa!".format(
                        type1, type2
                    )
                )


def refl_trans_closure(type_hierarchy):
    # make everything a set
    # and compute reflexive closure
    closure = defaultdict(set)
    for t in type_hierarchy:
        closure[t] = set(type_hierarchy[t])
        closure[t].add(t)

    # compute transitive closure
    still_changes = True
    while still_changes:
        still_changes = False
        for x in closure:
            new_for_x = set()
            for y in closure[x]:
                for z in closure[y]:
                    new_for_x.add(z)
            len_before = len(closure[x])
            closure[x].update(new_for_x)
            still_changes |= len(closure[x]) > len_before

    return closure


class HierarchicalFeature(Feature):
    def __init__(self, name, type_hierarchy, **kwargs):
        super(HierarchicalFeature, self).__init__(name, **kwargs)

        self.hierarchy = refl_trans_closure(type_hierarchy)
        check_sanity_constraints(self.hierarchy)

    def unify_base_values(self, fval1, fval2, bindings):
        candidates = self.hierarchy[fval1].intersection(self.hierarchy[fval2])
        score = {t: 0 for t in candidates}
        for type1, type2 in itertools.product(candidates, candidates):
            if type1 in self.hierarchy[type2]:
                score[type1] += 1

        return min(candidates, key=score.__getitem__, default=UnificationFailure)

--- 



[zurück zur Übersicht](#Übersicht)

---

## 1. Ambiguität in Konstituentengrammatik


### Gegeben sei folgender Satz: 

***time flies like an arrow***

### Schreiben Sie zu diesem *Beispielsatz für syntaktische Ambiguität*  eine minimale CFG, die genau die intendierte Struktur des Satzes erkennt.

### Folgende *intendierte Struktur* soll dabei gemäß Ihrer Gruppennummer modelliert werden:


#### `Gruppe 1` > ***time (flies like an arrow)VP***
#### `Gruppe 2` > ***time flies (like an arrow)VP***
#### `Gruppe 3` > ***(time flies like an arrow)VP***



### Testen Sie anschließend Ihre Grammatik (nur die erwünschte Analyse wird ausgegeben).


In [ ]:
#TO_DO:
sentence = "time flies like an arrow"

grammar = nltk.CFG.fromstring("""

""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (1):*
<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = "time flies like an arrow"

grammar = nltk.CFG.fromstring("""

""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)
</code>
</div>

---

## 2. Ambiguität in Dependenzgrammatik

### Gegeben sei wieder folgender Satz: 

***time flies like an arrow***


### Schreiben Sie nun eine minimale ungelabelte Dependenzgrammatik, die genau die intendierte Struktur des Satzes erkennt.

### Folgende *intendierte Struktur* soll dabei nun gemäß Ihrer Gruppennummer modelliert werden:

#### `Gruppe 1` > ***(time flies like an arrow)VP***
#### `Gruppe 2` > ***time (flies like an arrow)VP***
#### `Gruppe 3` > ***time flies (like an arrow)VP***


### Testen Sie anschließend Ihre Grammatik (nur die erwünschte Analyse wird ausgegeben).


### *Verwenden Sie die TIGER-Dependenzregeln!*








In [ ]:
#TO_DO:
sentence = "time flies like an arrow"

grammar = nltk.DependencyGrammar.fromstring("""

""")

parser = nltk.ProjectiveDependencyParser(grammar)
for tree in parser.parse(sentence.split()):
    print(tree, "\n")
    tree.pretty_print(unicodelines=True)

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = "time flies like an arrow"

grammar = nltk.DependencyGrammar.fromstring("""

""")

parser = nltk.ProjectiveDependencyParser(grammar)
for tree in parser.parse(sentence.split()):
    print(tree, "\n")
    tree.pretty_print(unicodelines=True)
</code>
</div>

--- 

## 3. Konstituenten- und Adjunkttests

## 3.1 Permutationstest

#### Gegeben sei folgender Satz sowie die Permutationen seiner Satzglieder:


In [2]:
#RUN (Führen Sie zunächst diese Code-Zelle aus):
sentence = ["bei ihm", "ist", "der Groschen", "gefallen"]

permutations = list(itertools.permutations(sentence))
for (i, item) in enumerate(permutations):
    print(i, item)    

0 ('bei ihm', 'ist', 'der Groschen', 'gefallen')
1 ('bei ihm', 'ist', 'gefallen', 'der Groschen')
2 ('bei ihm', 'der Groschen', 'ist', 'gefallen')
3 ('bei ihm', 'der Groschen', 'gefallen', 'ist')
4 ('bei ihm', 'gefallen', 'ist', 'der Groschen')
5 ('bei ihm', 'gefallen', 'der Groschen', 'ist')
6 ('ist', 'bei ihm', 'der Groschen', 'gefallen')
7 ('ist', 'bei ihm', 'gefallen', 'der Groschen')
8 ('ist', 'der Groschen', 'bei ihm', 'gefallen')
9 ('ist', 'der Groschen', 'gefallen', 'bei ihm')
10 ('ist', 'gefallen', 'bei ihm', 'der Groschen')
11 ('ist', 'gefallen', 'der Groschen', 'bei ihm')
12 ('der Groschen', 'bei ihm', 'ist', 'gefallen')
13 ('der Groschen', 'bei ihm', 'gefallen', 'ist')
14 ('der Groschen', 'ist', 'bei ihm', 'gefallen')
15 ('der Groschen', 'ist', 'gefallen', 'bei ihm')
16 ('der Groschen', 'gefallen', 'bei ihm', 'ist')
17 ('der Groschen', 'gefallen', 'ist', 'bei ihm')
18 ('gefallen', 'bei ihm', 'ist', 'der Groschen')
19 ('gefallen', 'bei ihm', 'der Groschen', 'ist')
20 ('gefal

### Führen Sie obenstehende Codezelle aus.

### Geben Sie (über den Listenindex) eine Permutation des Satzes an, welche folgendes Satzglied als Konstituente bestätigt und die gleichzeitig der *Nebensatzwortstellung* entspricht (Zuordnung gemäß Gruppennummer):


#### `Gruppe 1` > Subjekt
#### `Gruppe 2` > Adverbial
#### `Gruppe 3` > finites Verb


In [ ]:
#TO_DO:
list(itertools.permutations(sentence))[0]

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (3.1):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
list(itertools.permutations(sentence))[0]
</code>
</div>

---


## 3.2 Adjunkttest


#### Gegeben sei folgender Satz, dessen drittes Satzglied den Averbialsatz-Test nicht besteht:

In [3]:
sentence = ["sie", "fuhren", "um 9 Uhr", "los"]

sentence[0] + " " + sentence[1] +  " " + sentence[3] + ", während sie " + sentence[2] + " waren"

'sie fuhren los, während sie um 9 Uhr waren'

### Geben Sie (unter Erhalt der Wohlgeformtheit des Ausgangssatzes) ein alternatives drittes Satzglied an (an Stelle des TODO), so dass der Averbialsatz-Test erfolgreich ist.


In [4]:
#TO_DO:
sentence = ["sie", "fuhren", "TODO", "los"]

sentence[0] + " " + sentence[1] +  " " + sentence[3] + ", während sie " + sentence[2] + " waren"

'sie fuhren los, während sie TODO waren'

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (3.2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = ["sie", "fuhren", "TODO", "los"]

sentence[0] + " " + sentence[1] +  " " + sentence[3] + ", während sie " + sentence[2] + " waren"
</code>
</div>

---

## 4. CFG-Analysen

#### Gegeben sei folgender Satz und eine entsprechende Grammatik:


In [51]:
sentence = "die Frau wartet"

grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    VP  -> V
    NP  -> Det N

    Det -> "die"
    N   -> "Frau"
    V   -> "wartet"    
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

         S         
     ┌───┴─────┐    
     NP        VP  
 ┌───┴───┐     │    
Det      N     V   
 │       │     │    
die     Frau wartet



### 4a) Ergänzen Sie den Satz der Angabe, so dass er ein 


#### `Gruppe 1` > Kasusadverbial  (Adjunkt)
#### `Gruppe 2` > präpositionales Adverbial  (Adjunkt)
#### `Gruppe 3` > Präpositionalobjekt (Komplement)


### enthält (Zuordnung gemäß Gruppennummer).


In [ ]:
#TO_DO:
sentence = "die Frau wartet TODO"

### 4b) Erweitern Sie die Grammatik um entsprechende lexikalische und syntaktische Regeln für den erweiterten Satz.  Halten Sie sich dabei in den ergänzten verbalen Regeln an das X-Bar-Schema!

#### Verwenden Sie `VERBAL` als Symbol für `V'`

In [ ]:
#TO_DO:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    VP  -> V
    NP  -> Det N

    Det -> "die"
    N   -> "Frau"
    V   -> "wartet"    

###########ERGAENZTE REGELN:    
    
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (4a):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = "die Frau wartet TODO"
</code>
</div>

*Ursprüngliche Angabe (4b):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    VP  -> V
    NP  -> Det N

    Det -> "die"
    N   -> "Frau"
    V   -> "wartet"    

###########ERGAENZTE REGELN:    
    
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    
</code>
</div>

---

## 5. Dependenzanalyse

### Analysieren Sie die Dependenzbeziehungen Ihres Beispielsatzes aus der Aufgabe 4a) im UD-Schema. Verwenden Sie das aus der Vorlesung bekannte Format: 

- pro Zeile: `Position, Wort, Position des Kopfes, Dependenzrelation`
- Wurzelknoten: `Position des Kopfes` = 0, `Dependenzrelation` = ROOT


In [5]:
#TO_DO:
sent_nr = """
1 x 2 dep
2 y 0 ROOT
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)  

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

y(ROOT)
   │    
 x(dep)



---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (5):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sent_nr = """
1 x 2 dep
2 y 0 ROOT
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)  

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})
</code>
</div>

---

## 6. FCFG-Erweiterung: Kasus, Agreement, Subkategorisierung

#### Gegeben sei folgende Grammatik:

In [6]:
#RUN (Führen Sie zunächst diese Code-Zelle aus):
grammar = nltk.CFG.fromstring("""
    S    -> NP VP
    VP   -> V NP
    VP -> V NP NP
    NP   -> Pron    

    Pron -> "er" | "dir" |  "es"
    V    -> "schenkt" | "hilft" | "schenken" | "helfen" 
""")

parser = nltk.ChartParser(grammar,trace=0)

#### Die Grammatik erkennt u.a. folgende, grammatisch korrekte Sätze:

In [7]:
sentence = "er schenkt es dir"
for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

              S           
 ┌────────────┴───┐        
 │                VP      
 │      ┌─────────┼────┐   
 NP     │         NP   NP 
 │      │         │    │   
Pron    V        Pron Pron
 │      │         │    │   
 er  schenkt      es  dir 



In [8]:
sentence = "er hilft dir"
for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

       S           
 ┌─────┴────┐       
 │          VP     
 │     ┌────┴───┐   
 NP    │        NP 
 │     │        │   
Pron   V       Pron
 │     │        │   
 er  hilft     dir 



### 6a) Geben Sie einen nach den Regeln der deutschen Grammatik nicht-wohlgeformten Satz an, der fälschlicherweise von der oben angegebenen Grammatik erkannt wird, obwohl er gegen das folgende Constraint verstößt (Zuordnung gemäß Gruppennummer): 


#### `Gruppe 1` > Subkategorisierung (Anzahl der Argumente)
#### `Gruppe 2` > Objekt-Kasus
#### `Gruppe 3` > Subjekt-Verb-Agreement


In [ ]:
#TO_DO:
neg_sentence = ""
for tree in parser.parse(neg_sentence.split()):
    tree.pretty_print(unicodelines=True)    

---

[zurück zur Übersicht](#Übersicht)


### 6b) Erweitern Sie die angegebene Grammatik um die notwendigen Merkmale, um die Überproduktion aus 6a) zu verhindern.  Die korrekten Sätzen von oben sollen dabei weiter von der Grammatik erkannt werden (Codezelle `POSITIVTESTS`).

##### Die Erweiterung muss nur die Überproduktion bzgl. des gemäß Ihrer Gruppenzuordnung zu betrachtenden Constraints verhindern!



In [ ]:
#TO_DO:
gramstring = r"""
% start S

    S    -> NP VP
    VP   -> V NP
    VP -> V NP NP
    NP   -> Pron    

    Pron -> "er" | "dir" |  "es"
    V    -> "schenkt" | "hilft" | "schenken" | "helfen" 
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=1)

#NEGATIVBEISPIEL (neg_sentence aus 6a):
for tree in parser.parse(neg_sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

In [ ]:
#RUN (POSITIVTESTS):
grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

sentence = "er schenkt es dir"
for tree in parser.parse(sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    
sentence = "er hilft dir"
for tree in parser.parse(sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (6a):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
neg_sentence = ""
for tree in parser.parse(neg_sentence.split()):
    tree.pretty_print(unicodelines=True)    
</code>
</div>

*Ursprüngliche Angabe (6b):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
gramstring = r"""
% start S

    S    -> NP VP
    VP   -> V NP
    VP -> V NP NP
    NP   -> Pron    

    Pron -> "er" | "dir" |  "es"
    V    -> "schenkt" | "hilft" | "schenken" | "helfen" 
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=1)

#NEGATIVBEISPIEL (neg_sentence aus 6a):
for tree in parser.parse(neg_sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
</code>
</div>

---

## 7. Slash-Kategorien, Inversion, Wortstellung



#### Gegeben sei folgender Satz und eine FCFG, die die Subjekt-VP-Inversion für den Fragesatz sowie das Herausbewegen des Fragepronomens modelliert:


In [9]:
sentence = "woran glaubt er"

gramstring = r"""
% start S

    S[+QUEST] -> QPron S[+INV]/NP
    S[+INV]/?x -> VP/?x NP
    VP/?x -> V NP/?x
    NP/NP -> 

    NP -> Pron
    
    QPron   -> "woran"
    V   -> "glaubt"
    Pron   -> "er" 

"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

               S[+QUEST]                    
   ┌───────────────┴──────────┐              
   │                     S[+INV]/NP[]       
   │               ┌──────────┴─────────┐    
   │           VP[]/NP[]               NP[] 
   │      ┌────────┴──────────┐         │    
QPron[]  V[]              NP[]/NP[]   Pron[]
   │      │                   │         │    
 woran  glaubt               ...        er  



### Kopieren Sie die Regel(n) aus obenstehender Grammatik herunter, die das folgende implementiert/implementieren (Zuordnung gemäß Gruppennummer):



#### `Gruppe 1` > Subjekt-VP-Inversion
#### `Gruppe 2` > Gap-Introduction
#### `Gruppe 3` > Gap-Realisierung




In [ ]:
#TO_DO:
gramstring = r"""

"""

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (7):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
gramstring = r"""

"""
</code>
</div>

---

## 8. Syntaxregeln komplexer Sätze

## 8.1 CFG-Regeln für komplexe Sätze

#### Gegeben sei folgender Satz sowie eine kleine CFG-Grammatik, die u.a. diesen Satz erkennt:



In [10]:
sentence = "sie glaubt daran"

grammar = nltk.CFG.fromstring("""
    S    -> NP VP
    VP   -> V NP
    NP   -> PRON    

    PRON -> "sie" | "daran" | "dich" | "wer"
    V    -> "glaubt" | "kennt" | "kennen"
    PTCL -> "zu"
    COMP -> "dass"
    COMP ->
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    
    

       S             
 ┌─────┴─────┐        
 │           VP      
 │     ┌─────┴────┐   
 NP    │          NP 
 │     │          │   
PRON   V         PRON
 │     │          │   
sie  glaubt     daran



### 8.1 a) Passen Sie den Satz so an, dass er einen:

#### `Gruppe 1` > Subjektsatz
#### `Gruppe 2` > Infinitiv-Objektsatz
#### `Gruppe 3` > finiten Objektsatz


### enthält (Zuordnung gemäß Gruppennummer).

### Greifen Sie ausschließlich auf die in der Grammatik vorhandenen Lexeme zurück!


#### (Satzzeichen müssen nicht berücksichtigt werden!)


In [ ]:
#TO_DO:
sentence = " sie glaubt daran "

---

[zurück zur Übersicht](#Übersicht)


### 8.1 b) Erweitern Sie die Grammatik um entsprechende syntaktische Regeln für Ihren  Satz aus 8.1a.


- Verwenden Sie nur die bestehenden sowie ggf. `SBAR` und `Comp` als neue Nonterminale
- X-Bar-Schema ist nicht notwendig (orientieren Sie sich an den Penn-Treebank-Regeln für komplexe Sätze)

##### Beachten Sie die invertierte Wortstellung im Nebensatz (Verbendstellung).

In [428]:
#TO_DO:
grammar = nltk.CFG.fromstring("""
    S    -> NP VP
    VP   -> V NP
    NP   -> PRON    

    PRON -> "sie" | "daran" | "dich" | "wer"
    V    -> "glaubt" | "kennt" | "kennen"
    PTCL -> "zu"
    COMP -> "dass"
    COMP ->

###########ERGAENZTE REGELN:    
    
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (8.1a):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = " sie glaubt daran "
</code>
</div>

*Ursprüngliche Angabe (8.1b):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
grammar = nltk.CFG.fromstring("""
    S    -> NP VP
    VP   -> V NP
    NP   -> PRON    

    PRON -> "sie" | "daran" | "dich" | "wer"
    V    -> "glaubt" | "kennt" | "kennen"
    PTCL -> "zu"
    COMP -> "dass"
    COMP ->

###########ERGAENZTE REGELN:    
    
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    
</code>
</div>

---

## 8.2. Analysieren Sie die Dependenzbeziehungen des folgenden Satzes im UD-Schema (Zuordnung gemäß Gruppennummer):


#### `Gruppe 1`: *wer etwas wagt gewinnt*

#### `Gruppe 2`: *er wagt zu spielen*

#### `Gruppe 3`: *er wagt dass er spielt*


In [11]:
#TO_DO:
sent_nr = """
1 x 2 dep
2 y 0 ROOT
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)  

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

y(ROOT)
   │    
 x(dep)



---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (8.2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sent_nr = """
1 x 2 dep
2 y 0 ROOT
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)  

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})
</code>
</div>

---

## 9. Parsing-Algorithmen und Rekursionstypen

### 9.1 Ändern Sie in folgender Grammatik die Reihenfolge der Regeln, so dass der verwendete Shift-Reduce-Parser auch ohne Backtracking eine Ableitung findet:


In [12]:
#TO_DO:
sentence = "the old man the boat"

grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> Det N
    VP -> V NP

    Det -> 'the'
    Adj -> 'old'
    N -> 'man'
    N -> 'boat'
    N -> 'old'
    V -> 'man'
""")

parser = nltk.ShiftReduceParser(grammar,trace=1)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

Parsing 'the old man the boat'
    [ * the old man the boat]
    [ 'the' * old man the boat]
    [ Det 'old' * man the boat]
    [ Det Adj 'man' * the boat]
    [ Det Adj N 'the' * boat]
    [ Det Adj N Det 'boat' * ]


---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (9.1):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = "the old man the boat"

grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> Det N
    VP -> V NP

    Det -> 'the'
    Adj -> 'old'
    N -> 'man'
    N -> 'boat'
    N -> 'old'
    V -> 'man'
""")

parser = nltk.ShiftReduceParser(grammar,trace=1)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    
</code>
</div>

--- 

### 9.2 Geben Sie (genau) eine rekursive CFG-Regel an, die mit einem Recursive-Descent-Parser verarbeitet werden kann.


In [ ]:
#TO_DO:
grammar = nltk.CFG.fromstring("""

""")

print(grammar)

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (9.2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
grammar = nltk.CFG.fromstring("""

""")

print(grammar)
</code>
</div>

---

## 10. Unifikationsparsing und getypte Merkmalstrukturen





## 10.1 Unifikation und Merkmalsstrukturen

#### Gegeben seien folgende (unifizierende) Merkmalsstrukturen:

In [13]:
f1 = FeatStruct("[AGR=[NUM=[+PL],PERS=1]]")
f2 = FeatStruct("[]")
print(f1.unify(f2))

[       [ NUM  = [ PL = True ] ] ]
[ AGR = [                      ] ]
[       [ PERS = 1             ] ]



### Geben Sie eine Merkmalsstruktur `f2` an, die ***nicht*** mit `f1`unifiziert (print-Ausgabe: None).

In [ ]:
#TO_DO:
f1 = FeatStruct("[AGR=[NUM=[+PL],PERS=1]]")
f2 = FeatStruct("[]")
print(f1.unify(f2))

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (10.1):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
f1 = FeatStruct("[AGR=[NUM=[+PL],PERS=1]]")
f2 = FeatStruct("[]")
print(f1.unify(f2))
</code>
</div>

---


## 10.2 Unifikation mit Typen

#### Gegeben sei folgende Typhierarchie:

$$\bot \sqsubseteq \text{Genitiv}$$
$$\bot \sqsubseteq \text{nicht-Genitiv}$$
$$\text{nicht-Genitiv} \sqsubseteq \text{Nominativ-Akkusativ}$$
$$\text{nicht-Genitiv} \sqsubseteq \text{Dativ}$$
$$\text{Nominativ-Akkusativ} \sqsubseteq \text{Nominativ}$$
$$\text{Nominativ-Akkusativ} \sqsubseteq \text{Akkusativ}$$

####  Sie wird (mit abgekürzten Typnamen) durch das `*CASE*`-Feature implementiert:

In [14]:
#RUN (Führen Sie zunächst diese Code-Zelle aus):
type_hierarchy = {
    "nichtGen": ["NomAkk", "Dat"],
    "NomAkk": ["Nom", "Akk"],
    "Dat": [],
    "Akk": [],
    "Gen": [],
    "Nom": []
}
CASE = HierarchicalFeature("CASE", type_hierarchy)
reader = FeatStructReader(features=(CASE,))

### Führen Sie obenstehende Codezelle aus, um die Typhierarchie zu laden.

### Geben Sie zwei Merkmalstrukturen  `f2` und `f3` an, sodass gilt:

`f1` subsumiert `f2`, `f2` subsumiert `f3`.

In [ ]:
#TO_DO:
f1 = reader.fromstring("[*CASE*=nichtGen]")
f2 = reader.fromstring("[]")
f3 = reader.fromstring("[]")
print(f1.subsumes(f2), f2.subsumes(f3)) 

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (10.2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
f1 = reader.fromstring("[*CASE*=nichtGen]")
f2 = reader.fromstring("[]")
f3 = reader.fromstring("[]")
print(f1.subsumes(f2), f2.subsumes(f3)) 
</code>
</div>

---

## 11. Statistisches Parsing


## 11.1 PCFG: Gewichte und Ableitungswahrscheinlichkeit



#### Gegeben sei folgende Mini-Treebank aus NP-Fragmenten mit Koordinationsambiguität:


In [326]:
treestrings = [
"(NP kleine (NP Hunde (CONJ und) Katzen))",
"(NP (NP kleine Hunde) (CONJ und) (NP Katzen))",
]

trees = []
for treestring in treestrings:
    trees.append(Tree.fromstring(treestring))
    
for tree in trees:
    tree.pretty_print(unicodelines=True)

              NP            
  ┌───────────┴────┐         
  │                NP       
  │      ┌─────────┼─────┐   
  │      │         │    CONJ
  │      │         │     │   
kleine Hunde     Katzen und 

             NP             
        ┌────┴────┬─────┐    
        NP       CONJ   NP  
  ┌─────┴────┐    │     │    
kleine     Hunde und  Katzen



### Passen Sie diese Mini-Treebank durch Duplizieren einer minimalen Anzahl an NP-Strukturen so an, dass die daraus induzierte Grammatik einen engen Skopus des Adjektivs (`kleine Hunde`) bevorzugt:


In [15]:
#TO_DO:
treestrings = [
"(NP kleine (NP Hunde (CONJ und) Katzen))",
"(NP (NP kleine Hunde) (CONJ und) (NP Katzen))",
    
]


trees = []
for treestring in treestrings:
    trees.append(Tree.fromstring(treestring))
    
    
#grammar induction:    
productions = []
S = nltk.Nonterminal('NP')

for tree in trees:
    productions += tree.productions()

grammar = nltk.induce_pcfg(S, productions)
for production in grammar.productions():
    print(production)    
    
#parse trees with grammar:    
parser = nltk.ViterbiParser(grammar)
     
for tree in trees:
    for parse in parser.parse(tree.leaves()): 
        print(parse)
        parse.pretty_print(unicodelines=True) 

NP -> 'kleine' NP [0.2]
NP -> 'Hunde' CONJ 'Katzen' [0.2]
CONJ -> 'und' [1.0]
NP -> NP CONJ NP [0.2]
NP -> 'kleine' 'Hunde' [0.2]
NP -> 'Katzen' [0.2]
(NP kleine (NP Hunde (CONJ und) Katzen)) (p=0.04)
              NP            
  ┌───────────┴────┐         
  │                NP       
  │      ┌─────────┼─────┐   
  │      │         │    CONJ
  │      │         │     │   
kleine Hunde     Katzen und 

(NP kleine (NP Hunde (CONJ und) Katzen)) (p=0.04)
              NP            
  ┌───────────┴────┐         
  │                NP       
  │      ┌─────────┼─────┐   
  │      │         │    CONJ
  │      │         │     │   
kleine Hunde     Katzen und 



---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (11.1):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
treestrings = [
"(NP kleine (NP Hunde (CONJ und) Katzen))",
"(NP (NP kleine Hunde) (CONJ und) (NP Katzen))",
    
]


trees = []
for treestring in treestrings:
    trees.append(Tree.fromstring(treestring))
    
    
#grammar induction:    
productions = []
S = nltk.Nonterminal('NP')

for tree in trees:
    productions += tree.productions()

grammar = nltk.induce_pcfg(S, productions)
for production in grammar.productions():
    print(production)    
    
#parse trees with grammar:    
parser = nltk.ViterbiParser(grammar)
     
for tree in trees:
    for parse in parser.parse(tree.leaves()): 
        print(parse)
        parse.pretty_print(unicodelines=True) 
</code>
</div>

---

## 11.2 Übergangsbasierter Shift-Reduce-Dependency-Parser

#### Gegeben sei folgender Dependenzgraph:

In [18]:
sent_nr = """
1 a 0 ROOT 
2 b 1 RIGHTARC-?
3 c 4 LEFTARC-?
4 d 1 RIGHTARC-?
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

### Geben Sie eine deutschen oder englischen Satz an, der diese Dependenzstrukturanalyse erfüllt; geben Sie außerdem die Reihenfolge der Durchführung der REDUCE-Übergänge mit einem Shift-Reduce-Dependency-Parser an. 

#### (Ersetzen Sie ausschließlich die Buchstaben `a,b,c,d` mit entsprechenden Wörtern sowie das Fragezeichen mit der Angaben der Nummer der Durchführung des REDUCE-Schritts.)

In [19]:
#TO_DO:
sent_nr = """
1 a 0 ROOT 
2 b 1 RIGHTARC-?
3 c 4 LEFTARC-?
4 d 1 RIGHTARC-?
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (11.2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sent_nr = """
1 a 0 ROOT 
2 b 1 RIGHTARC-?
3 c 4 LEFTARC-?
4 d 1 RIGHTARC-?
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})
</code>
</div>

---

## 12. Datengestützte Syntaxanalyse




## 12.1 Lexikalisierte CFG (mit Merkmalen)

#### Gegeben sei folgende FCFG:

In [20]:
sentence = "er erklimmt den Berg"

gramstring = r"""
% start S
    S[]   -> NP[] VP[]
    VP[]  -> V[] NP[]
    NP[]  -> Det[] N[]
    NP[]  -> N[]

    Det[] -> "den"
    N[]   -> "er"    
    N[]   -> "Berg"
    V[]   -> "erklimmt"    
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    print(tree)    
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

(S[]
  (NP[] (N[] er))
  (VP[] (V[] erklimmt) (NP[] (Det[] den) (N[] Berg))))
              S[]                
 ┌─────────────┴────┐             
 │                 VP[]          
 │      ┌───────────┴────┐        
NP[]    │               NP[]     
 │      │           ┌────┴────┐   
N[]    V[]        Det[]      N[] 
 │      │           │         │   
 er  erklimmt      den       Berg



### Führen Sie über ein HEAD-Merkmal in der FCFG eine vollständige Kopfannotation durch.

#### (Sie können ggf. sowohl die UD als auch die TIGER-Dependenzregeln verwenden.)


In [ ]:
#TO_DO:
sentence = "er erklimmt den Berg"

gramstring = r"""
% start S
    S[]   -> NP[] VP[]
    VP[]  -> V[] NP[]    
    NP[]  -> Det[] N[]
    NP[]  -> N[]

    Det[] -> "den"
    N[]   -> "er"    
    N[]   -> "Berg"
    V[]   -> "erklimmt"    
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    print(tree)    
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (12.1):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = "er erklimmt den Berg"

gramstring = r"""
% start S
    S[]   -> NP[] VP[]
    VP[]  -> V[] NP[]    
    NP[]  -> Det[] N[]
    NP[]  -> N[]

    Det[] -> "den"
    N[]   -> "er"    
    N[]   -> "Berg"
    V[]   -> "erklimmt"    
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    print(tree)    
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
</code>
</div>


---


## 12.2 Parent Annotation (mit Symbolerweiterung)

#### Gegeben sei wieder folgende CFG:


In [22]:
sentence = "er erklimmt den Berg"

grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    VP  -> V NP
    NP  -> Det N
    NP  -> N

    Det -> "den"
    N   -> "er"    
    N   -> "Berg"
    V   -> "erklimmt"        
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

              S              
 ┌────────────┴───┐           
 │                VP         
 │     ┌──────────┴───┐       
 NP    │              NP     
 │     │          ┌───┴───┐   
 N     V         Det      N  
 │     │          │       │   
 er erklimmt     den     Berg



### Führen Sie über Symbolerweiterung (mit `^` als Trennerzeichen) in der CFG eine vollständige *Parent Annotation* durch, wie Sie durch die Regelanwendungen im Syntaxbaum der Angabe impliziert ist. 

##### Ergänzen Sie ggf. auch Regeln, die für eine vollständige Parent-Annotation notwendig sind.

In [ ]:
#TO_DO:
sentence = "er erklimmt den Berg"

grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    VP  -> V NP
    NP  -> Det N
    NP  -> N

    Det -> "den"
    N   -> "er"    
    N   -> "Berg"
    V   -> "erklimmt"    

###########ERGAENZTE REGELN:    
    
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (12.2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = "er erklimmt den Berg"

grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    VP  -> V NP
    NP  -> Det N
    NP  -> N

    Det -> "den"
    N   -> "er"    
    N   -> "Berg"
    V   -> "erklimmt"    

###########ERGAENZTE REGELN:    
    
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    
</code>
</div>

---

## 13. Chunk-Analysen



## 13.1 Geben Sie einen folgende *IOB-Tag-Sequenz* einer VP-NP-Chunk-Analyse erfüllenden deutschen Satz an, indem Sie die TODOs mit entsprechenden Strings ersetzen (nehmen Sie keine weiteren Änderungen vor).

#### (Beachten Sie auch die gegebenen Satzzeichen bzgl. möglicher Satztypen und entsprechender Worstellung.)


In [24]:
#TO_DO:
iob_list = [
("B-VP", "TODO"), 
("B-NP", "TODO"),
("B-NP", "TODO"),
("I-NP", "TODO"),
("O", ","),
("O", "TODO"),    
("B-NP", "TODO"),
("B-NP", "TODO"),
("B-VP", "TODO"),
("O", "!")
]

iob_list

[('B-VP', 'TODO'),
 ('B-NP', 'TODO'),
 ('B-NP', 'TODO'),
 ('I-NP', 'TODO'),
 ('O', ','),
 ('O', 'TODO'),
 ('B-NP', 'TODO'),
 ('B-NP', 'TODO'),
 ('B-VP', 'TODO'),
 ('O', '!')]

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (13.1):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
iob_list = [
("B-VP", "TODO"), 
("B-NP", "TODO"),
("B-NP", "TODO"),
("I-NP", "TODO"),
("O", ","),
("O", "TODO"),    
("B-NP", "TODO"),
("B-NP", "TODO"),
("B-VP", "TODO"),
("O", "!")
]

iob_list
</code>
</div>